In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns 

%matplotlib inline

## Part 0: Loading and cleaning the data

In [ ]:
data_path = "CrowdstormingDataJuly1st.csv"
df = pd.read_csv(data_path)

**Each row in the dataset is a (player _p_, referee _r_) dyad containing some information about the player, total number of each type of card (yellow, yellow-red, and red) given by _r_ to _p_, and some statistics about racial bias in the referee's home country.**

In [ ]:
df.sample(5)

In [ ]:
print("Number of entries: %d" % len(df))

**We want to make sure we do our analysis with a clean dataset, so we check for null entries**

In [ ]:
df.isnull().sum()

**We can see that there are many players without skin color ratings. Those players aren't going to be useful for our prediction task, so we drop all of them**

In [ ]:
df = df.dropna(subset=["rater1", "rater2"])
print(len(df))

**What other missing values are we dealing with in this dataset?**
- Position: We see that of over 100,000 dyads there are several thousand rows missing the position field. We choose not to modify these values because we aren't going to use the player's position for our prediction.
- Height/weight and bias information for referee country: only a few hundred rows are missing this information, so we choose to drop all.

In [ ]:
df = df.dropna(subset=['height', 'weight', 'Alpha_3', 'meanIAT', 'nIAT', 'seIAT', 'meanExp', 'nExp', 'seExp'])
df.isnull().sum()

In [ ]:
len(df)

**OK, now we have a clean dataset to work with!**

**We read through the Crowdstorming analytics visualization notebook linked in the homework description. Unlike them, we decided that we do not have a strong reason to drop the dyads that they judged "incomplete" (i.e. did not have at least 11 referee-player dyads from the same club).**

**We decided to only work with the numeric features in the dataset, such as:
height, weight, game statistics, goals, cards, skin color ratings, and referee statistics.
We may be able to achieve better accuracy by encoding categorical features like the position and the country but for this assignment we chose to focus on playing with the classifier rather than going for maximum accuracy**

In [ ]:
# How we apparently compute our features:
# Group by player, take the mean over the dyad
# This doesn't make much sense for total game stats,
# but the average number goals per game, cards per game, 
# and most importantly meanIAT and meanExp of the referees
# over all the player's games are useful
df_by_player = df.groupby("playerShort")
df_players = df_by_player.agg(np.mean)
df_players.head()

In [ ]:
df_players.describe()


In [ ]:
# Create a df with player constant description attributes
df_players_description = pd.DataFrame(df_players[["height", "weight", "rater1", "rater2"]])
df_players_description.sample(5)
len(df_players_description)

# Part 1: Random Forest

## Raters consistency

We suspect that the raters have a certain bias and do not always rate the same player the same way. We look at the differences

In [ ]:
(df_players_description["rater1"] - df_players_description["rater2"]).describe()

We see that rater2 rates the skintone higher than rater1 on average. 
We now make a new attribute that is the mean of rater1 and rater2's scores. 

In [ ]:
df_players_description["rateMean"] = (df_players_description["rater1"] + df_players_description["rater2"]) / 2

In [ ]:
df_players_description.sample(5)

Since random forest uses binary classification, we decided to define a binary attribute "darkSkin". We have to choose the limit between "white" and "black". We arbitrarily chose a mean rating greater than or equal to 0.5 to be considered "darkSkin".

In [ ]:
df_players_description['darkSkin'] = df_players_description['rateMean']  >= 0.5
df_players_description.head(10)

In [ ]:
print(df_players_description.isnull().sum())

In [ ]:
df_players_description =  df_players_description.dropna()

## Random forest machine learning

**First we train a RandomForestClassifier that given almost all of the features (except skin color ratings of course) outputs his skin color. We'll show that using the default settings it overfits to the training set, then try again with cross-validation**

In [ ]:
df_players.head()
df_players['darkSkin'] = df_players_description['darkSkin']

X_all = df_players.drop(['rater1', 'rater2', 'darkSkin'], axis=1) # not in-place
y_all = df_players_description['darkSkin']


In [ ]:
X_all.head()

In [ ]:
from sklearn import metrics
from sklearn.cross_validation import train_test_split

def print_metrics(y, y_pred):
    # Print out some metrics judging the quality of the classification
    print("Accuracy:")
    print(metrics.accuracy_score(y, y_pred))
    print("F1: ")
    print(metrics.f1_score(y, y_pred))
    print("Confusion matrix:")
    print(metrics.confusion_matrix(y, y_pred))
    print("-------")

# Split the data into 60% of training set, and 40% of test set
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.4)

clf_all = RandomForestClassifier()
clf_all.fit(X_train, y_train)
y_pred_train = clf_all.predict(X_train)
y_pred_test = clf_all.predict(X_test)
print('Accuracy on training set is {0:.2f}%'.format(metrics.accuracy_score(y_train, y_pred_train)*100))
print_metrics(y_train, y_pred_train)
print('Accuracy on testing set is {0:.2f}%'.format(metrics.accuracy_score(y_test, y_pred_test)*100))
print_metrics(y_test, y_pred_test)

**So, we obtain a nearly perfect classification accuracy on the training set but much worse on the test set. This isn't too surprising. With our small amount of data the random forest is likely overfitting to the jumble of numeric features we gave it. We will try reducing the maximum allowed height of the trees and see how the accuracy changes:**
- max_depth: parameter governing height of the trees

In [ ]:
clf_shorter = RandomForestClassifier(max_depth=5)

clf_shorter.fit(X_train, y_train)
y_pred_train_shorter = clf_shorter.predict(X_train)
y_pred_test_shorter = clf_shorter.predict(X_test)

print('Accuracy on training set is {0:.2f}%'.format(metrics.accuracy_score(y_train, y_pred_train_shorter)*100))
print_metrics(y_train, y_pred_train_shorter)
print('Accuracy on testing set is {0:.2f}%'.format(metrics.accuracy_score(y_test, y_pred_test_shorter)*100))
print_metrics(y_test, y_pred_test_shorter)

**When we only allow short trees (max_depth <= 5) in our random forest it seems to reduce the overfitting issue, based on the improved accuracy and F1 score.**

**Next part: We start again with random forest, this time using cross validation.
Let's start with a simple model: considering only height and weight, try to obtain the player's skin color.**

In [ ]:
X = df_players_description[['height', 'weight']]
y = df_players_description['darkSkin']
clf = RandomForestClassifier(n_estimators=100)

In [ ]:
clf.fit(X,y)
y_pred = clf.predict(X)
print('The accuracy is {0:.2f}%'.format(metrics.accuracy_score(y, y_pred)*100))

  81% accuracy can seem pretty good, but we must remember that we are training on the whole dataset so it doesn't mean much. If we would try to predict on unseen data, the result would be poor as we are probably overfitting the training set. Now let's use cross-validation!

**Note: one example with train-test split**

In [ ]:
from sklearn.cross_validation import train_test_split

# Split the data into 60% of training set, and 40% of test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print('The accuracy is {0:.2f}%'.format(metrics.accuracy_score(y_test, y_pred)*100))

Training on 60% of the data gives around 70% accuracy. Let's try with a 20-fold cross validation.

In [ ]:
from sklearn.cross_validation import cross_val_score

clf = RandomForestClassifier(n_estimators=100)
scores = cross_val_score(clf, X, y, cv=20, scoring='accuracy')
pd.Series(scores).describe()

Median score is 72%, and the standard deviation is not too high. That a decent score, considering that we are only looking at two features: height and weight. Let's try to add game features and see how the score changes.

In [ ]:
df_players.head(5)

In [ ]:
def random_forest_scores(dataframe, features, target="darkSkin", estimators=100, folds=20):
    clf = RandomForestClassifier(n_estimators=estimators)
    X = dataframe[features]
    y = list(dataframe["darkSkin"].values)
    
    # Cross validation scores
    scores = cross_val_score(clf, X, y, cv=folds, scoring='accuracy')
    
    # Train again to get feature importances
    X_train, _, y_train, _ = train_test_split(X, y, test_size=0.4)
    clf.fit(X_train, y_train)
    
    return scores, clf.feature_importances_

In [ ]:
df_players = df_players.dropna()

In [ ]:
possible_features = ['height', 'weight', 'games', 'victories', 'ties', 'defeats', 'goals', 'yellowCards', 'yellowReds', 'redCards']

In [ ]:
scores, importances = random_forest_scores(df_players, possible_features)

In [ ]:
pd.Series(scores).describe()

Adding features adds around 3% of precision.

In [ ]:
def plot_feature_importances(possible_features, importances):
    df_feature_importances = pd.DataFrame({"features": possible_features, "importances": importances})
    df_feature_importances = df_feature_importances.set_index("features")
    df_feature_importances = df_feature_importances.sort_values("importances", ascending=False)
    df_feature_importances.plot(kind="bar")

In [ ]:
plot_feature_importances(possible_features, importances)

In [ ]:
possible_features_2 = list(possible_features)
possible_features_2.remove("goals")

In [ ]:
scores2, importances2 = random_forest_scores(df_players, possible_features_2)

In [ ]:
pd.Series(scores2).describe()

In [ ]:
pd.Series(scores).describe()

**Let's now try to incorporate the information about racial bias in each referee's country of origin.**

Previously we computed the average meanIAT and meanExp values for each player over all referee-player dyads. We again use the existing meanIAT and meanExp information from each dyad to create two new features for each player: stdIAT and stdExp. Note that this is not the same calculation as averaging the standard deviations _seIAT_ and _seExp_ from each dyad - on a high-level stdIAT and stdExp indicate the variance in referee nationalities (and potentially biases) over  all matches in which the player played.

This gives us four racial bias-related features:
- meanIAT = average of meanIAT values over all dyads containing the given player
- meanExp = average of meanExp values over all dyads containing the given player
- stdIAT = std deviation of meanIAT values over all dyads containing the given player. General idea: has this player been referee'd by a diverse group of referees?
- stdExp = std deviation of meanExp values over all dyads containing the given player

**We also computed a meanIAT_carded features, which is the average of meanIAT values over all dyads where the player received at least one card from that referee. But we commented it out because it didn't seem to work well**

In [ ]:
#Not going to use this feature after all (meanIAT_carded)
#df['meanIAT_carded'] = df.apply(lambda x: x.meanIAT*(x.yellowCards+x.yellowReds+x.redCards), axis=1)
#df_players[['stdIAT','stdExp', 'stdIAT_carded']] = df_by_player['meanIAT', 'meanExp', 'meanIAT_carded'].agg(np.std)
df_players[['stdIAT','stdExp']] = df_by_player['meanIAT', 'meanExp'].agg(np.std)

**We check that our new standard deviation features were computed correctly**

In [ ]:
df_players.sample(5)

In [ ]:
df_players[df_players['stdExp'].isnull()]#.sum()

In [ ]:
df_players[df_players['stdIAT'].isnull()]#.sum()

**We can see that there are some NaN values so we replace them with 0**

In [ ]:
df_players.replace(to_replace={'stdIAT': {np.nan : 0}, 'stdExp': {np.nan : 0}, 'stdIAT_carded': {np.nan : 0}}, inplace=True)

In [ ]:
#possible_features_with_bias = ['stdIAT_carded', 'meanIAT_carded', 'meanIAT', 'stdIAT', 'meanExp', 'stdExp', 'height', 'weight', 'games', 'victories', 'ties', 'defeats', 'goals', 'yellowCards', 'yellowReds', 'redCards']
possible_features_with_bias = ['meanIAT', 'meanExp', 'height', 'weight', 'games', 'victories', 'ties', 'defeats', 'goals', 'yellowCards', 'yellowReds', 'redCards']

In [ ]:
scores_bias, importances_bias = random_forest_scores(df_players, possible_features_with_bias)

In [ ]:
pd.Series(scores_bias).describe()

In [ ]:
plot_feature_importances(possible_features_with_bias, importances_bias)

**When we include the features relating to the racial bias in referee countries, we see a bump of several percent in accuracy, up to 78-79%. These features are ranked highly in importance so they seem to be quite important to the prediction!**

# Part 2: Clustering

We choose KMeans as clustering algorithm as it is the simplest.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

We use the features corresponding to national racial bias in the referee's home countries, since in part 1 we found these to be predictive of skin color.

In [ ]:
bias_features = ['meanIAT', 'stdIAT', 'meanExp', 'stdExp']

In [ ]:
X = df_players[bias_features]

We run KMeans with the default k-means++ initialization strategy. We're looking for two clusters corresponding to dark skinned and light skinned players respectively.

In [ ]:
kmeans = KMeans(n_clusters=2).fit(X)
cluster_indexes = kmeans.predict(X)

In [ ]:
cluster1 = df_players.loc[cluster_indexes == 0]
cluster2 = df_players.loc[cluster_indexes == 1]

def dark_skin_proportion(cluster):
    """Return the proportion of dark skin player in the cluster"""
    return cluster['darkSkin'].sum() / len(cluster)

In [ ]:
print(dark_skin_proportion(cluster1), dark_skin_proportion(cluster2))

In [ ]:
def cluster_dark_separator_score(cluster1, cluster2):
    """Returns a heuristic that gives a score between 0 and 1 to the cluster assignment.
    The higher the score, the more the clusters separate players by skin colors"""
    return np.abs(dark_skin_proportion(cluster1) - dark_skin_proportion(cluster2))

In [ ]:
cluster_dark_separator_score(cluster1, cluster2)

In [ ]:
def kmean_dark_separator_score(X):
    """Given a dataset, returns heuristic score and silhouette score for the
    clusters generated by KMeans"""
    kmeans = KMeans(n_clusters=2).fit(X)
    cluster_indexes = kmeans.predict(X)
    cluster1 = df_players.loc[cluster_indexes == 0]
    cluster2 = df_players.loc[cluster_indexes == 1]
    
    
    return cluster_dark_separator_score(cluster1, cluster2), silhouette_score(X, cluster_indexes)

### Maximize score

Let's look at all 1-feature cases

In [ ]:
feature_score_silhouette = []

for feature in possible_features_with_bias:
    score, silhouette = kmean_dark_separator_score(df_players[[feature]])
    feature_score_silhouette.append((feature, score, silhouette))

We can see that for the 1-feature case, the silhouette is always between 0.5-0.75 which is quite good. We thus order by the score they give when they are used alone in KMeans.

In [ ]:
feature_score_silhouette = sorted(feature_score_silhouette, key=lambda x:x[1])
["Feature: {:12} Score: {:.2f}    Silhouette: {:.2f}".format(feature, score, silhouette) 
     for feature, score, silhouette in reversed(feature_score_silhouette)]

We order the features by score they give as a single KMean feature

In [ ]:
ordered_features = [fscore[0] for fscore in feature_score_silhouette]
",".join(ordered_features)

We remove each feature iteratively and print the resulting heuristic score and silhouette score

In [ ]:
# Remove each feature iteratively
for i in range(len(ordered_features)):
    score, silhouette = kmean_dark_separator_score(df_players[ordered_features[i:]])
    
    print('Before removing {:12} score is {:.02f}  silhouette is {:.02f}'.format(ordered_features[i], score, silhouette))

We see that keeping only `["meanIAT", "stdIAT"]` is optimal

In [ ]:
X = df_players[["meanIAT", "stdIAT"]]
kmeans = KMeans(n_clusters=2).fit(X)
cluster_indexes = kmeans.predict(X)
cluster1 = df_players.loc[cluster_indexes == 0]
cluster2 = df_players.loc[cluster_indexes == 1]

FMT = "Cluster {}: {:.2f}% of dark skin players"
print(FMT.format(1, 100 * dark_skin_proportion(cluster1)))
print(FMT.format(2, 100 * dark_skin_proportion(cluster2)))

We see the clustering works decently, one cluster contains **a vast majority of dark skin players** while the other contains a **vast majority of light skin players**.

Also, the silhouette score is good.

In [ ]:
"Silhouette score: {:.2f}".format(silhouette_score(X, cluster_indexes))

**We thus find that using features `["meanIAT", "stdIAT"]` gives two clusters with good silhouette score, and separates quite well dark-skin and light-skin players**